# 第27章预训练语言模型

## 习题27.1
&emsp;&emsp;设计基于双向LSTM的预训练语言模型，假设下游任务是文本分类。

**解答：**  

**解答思路：**

**解答步骤：**  

## 习题27.2
&emsp;&emsp;假设GPT微调的下游任务是两个文本的匹配，写出学习的目标函数。

**解答：**

**解答思路：**  

**解答步骤：**   

## 习题27.3
&emsp;&emsp;设计一个2层卷积神经网络编码器和2层卷积神经网络解码器组成的自动编码器（使用第28章介绍的转置卷积）。

**解答：**

**解答思路：**  

**解答步骤：**   

## 习题27.4
&emsp;&emsp;证明当编码器和解码器都是线性函数时，主成分分析可以作为自动编码器学习的方法。

**解答：**

**解答思路：**  

**解答步骤：**   

## 习题27.5
&emsp;&emsp;解释为什么BERT预训练中的掩码语言模型化是基于去噪自动编码器原理的。

**解答：**

**解答思路：**  

**解答步骤：**   

## 习题27.6
&emsp;&emsp;比较BERT与Transformer编码器在模型上的异同。

**解答：**

**解答思路：**  

**解答步骤：**   